
 **Import necessary libraries**



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import os

# List all files in the current directory
print(os.listdir('.'))


['.config', 'sample_data']


*** Load and preprocess the CIFAR-10 dataset**

In [ ]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


**the CNN model**

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Compile the model**

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


**Train the model**

In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.4586 - loss: 1.5484 - val_accuracy: 0.5810 - val_loss: 1.1827
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.6570 - loss: 0.9759 - val_accuracy: 0.6124 - val_loss: 1.1421
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7109 - loss: 0.8169 - val_accuracy: 0.6721 - val_loss: 0.9540
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.7480 - loss: 0.7126 - val_accuracy: 0.6464 - val_loss: 1.0454
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7786 - loss: 0.6381 - val_accuracy: 0.6978 - val_loss: 0.9329
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7975 - loss: 0.5782 - val_accuracy: 0.7252 - val_loss: 0.8215
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8206 - loss: 0.5121 - val_accuracy: 0.6965 - val_loss: 0.9400
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8385 - loss: 0.464

In [ ]:
# Save the trained model
model.save("cifar10_cnn_model.h5")

**Download the Model from Colab**

In [ ]:
from google.colab import files

# Download the model
files.download('cifar10_cnn_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**FastAPI**

Install FastAPI and Uvicorn:

In [ ]:
pip install fastapi uvicorn tensorflow pillow


Create the FastAPI Application:

In [ ]:
from fastapi import FastAPI, UploadFile, File
import uvicorn
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

# Load the trained model
model = load_model("cifar10_cnn_model.h5")

app = FastAPI()

def preprocess_image(image: Image.Image):
    image = image.resize((32, 32))
    image = np.array(image).astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    return image

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    image = Image.open(file.file)
    processed_image = preprocess_image(image)
    prediction = model.predict(processed_image)
    predicted_class = np.argmax(prediction, axis=1)[0]
    return {"predicted_class": int(predicted_class)}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


Run the FastAPI Application:

In [ ]:
uvicorn app:app --reload


Deploy FastAPI on a Cloud Service

Example: Deploying on Heroku
Install the Heroku CLI and log in:

**code**

heroku login

Create a Procfile in your project directory:


**code**

web: uvicorn your_fastapi_file_name:app --host=0.0.0.0 --port=${PORT:-5000}

Create a requirements.txt:

**code**

pip freeze > requirements.txt
Initialize a Git repository and push to Heroku:

**code**

git init
git add .
git commit -m "Initial commit"
heroku create your-app-name
git push heroku master


Open the Heroku app:

**code**

heroku open